In [4]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PROJ_LIB'] = r'C:/Users/mastr/miniconda3/pkgs/proj4-5.2.0-ha925a31_1/Library/share'     ## Windows OS
# os.environ['PROJ_LIB'] = r'/Users/mmastro/miniconda3/pkgs/proj4-5.2.0-ha925a31_1/Library/share'     ## Mac OS
import glob
import netCDF4 as nc
import numpy as np
import pandas as pd
import xarray as xr
#from scipy.signal import argrelextrema                      # Find local Maxima-Minima in numpy array
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy as cart
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

In [5]:
## Function for getting the indeces of the array *arr* between the *var_min* and *var_max* values:
def getRangeIndexes(arr, var_min, var_max):
    return np.where((np.array(arr) >= var_min) & (np.array(arr) <= var_max))[0]

## Function for getting the index of the array for the value ##
def getIndexes(arr, value):
    return np.where(arr = value)

## Function for moving average ##
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

## Fuction for subsetting colormap values ## 
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

## -- Function for get strings of dates in .nc input files -- ##
def set_string_time(file, tname):
    nctime = file.variables[tname][:] # get values
    t_unit = file.variables[tname].units # get unit  "days since 1950-01-01T00:00:00Z"
    t_cal = file.variables[tname].calendar
    tvalue = nc.num2date(nctime,units = t_unit,calendar = t_cal)
    str_time = [i.strftime("%Y-%m") for i in tvalue]
    str_timey = [i.strftime("%Y") for i in tvalue]
    str_timem = [i.strftime("%m") for i in tvalue]
    return [str_time, str_timey, str_timem]

## Function for creating a path, if needed ##
def checkDir(out_path):
    if not os.path.exists(out_path):
        os.makedirs(out_path)


## Open SST data

In [6]:
##### ------- Open data (MODEL) ------- #####
data_path = 'C:/Users/mastr/Documents/Amazon'
scenario = 'historical'
files = "nino34" + '_*_' + scenario + '_*'

files_list = []; esm = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/ENSO/detrended/'+files))):               # List of files sorted by name
        esm.append(filepath.split("_")[1])
        content = nc.Dataset(filepath)
        files_list.append(content)   

nino34_hist = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/ENSO/detrended/'+files))):               # sorted is case sensitive                             ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"]); #content = content.mean(dim="time")         ## values   var     dims    coords
    nino34_hist.append(content)

scenario = 'ssp585'
files = "nino34" + '_*_' + scenario + '_*' 

nino34_ssp = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/ENSO/detrended/'+files))):               # sorted is case sensitive                             ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"]); #content = content.mean(dim="time")         ## values   var     dims    coords
    nino34_ssp.append(content)

# Standardize calendar 
# for i, item in enumerate(nino34_hist):
#     if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
#         nino34_hist[i]['time'] = item.indexes['time'].to_datetimeindex()
    
# for i, item in enumerate(nino34_ssp):
#     if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
#         nino34_ssp[i]['time'] = item.indexes['time'].to_datetimeindex()

## Normalization
for i,item in enumerate(nino34_hist):
    nino34_hist[i] =  ((nino34_hist[i] - (nino34_hist[i].mean(dim='time'))).compute()/(nino34_hist[i].std(dim='time'))).compute()

for i,item in enumerate(nino34_ssp):
    nino34_ssp[i] =  ((nino34_ssp[i] - (nino34_ssp[i].mean(dim='time'))).compute()/(nino34_ssp[i].std(dim='time'))).compute()


# # Detrending of 1st order
# nino34_hist = [detrend_dim(a.tos, "time", 1) for a in nino34_hist]
# nino34_ssp = [detrend_dim(a.tos, "time", 1) for a in nino34_ssp]

## Resample from Monthly to seasonal timesteps
nino34_hist_seas = []
for i,item in enumerate(nino34_hist):
    content = item.resample(time="QS", label='left').mean()
    nino34_hist_seas.append(content)

nino34_ssp_seas = []
for i,item in enumerate(nino34_ssp):
    content = item.resample(time="QS", label='left').mean()
    nino34_ssp_seas.append(content)

In [7]:
for i, item in enumerate(reversed(esm)):
    if item in {"BCC-CSM2-MR","E3SM-1-1-ECA","TaiESM1","NorESM2-MM"}:
        del nino34_hist_seas[i]
        del nino34_ssp_seas[i]

## Open LAND data

To perform a composite analysis and assess the Mean State Changes (MSC) in the ssp585 compared to the historical period, we need to **remove te eCO2 effect on NEP/NBP from future scenario only** and compare theresulting NEP_net/NBP_net with the ones from the historical period 

In [8]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'nep'
files = var_name + '_*_historical_'  + '*.nc' 


files_list = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):               # List of files sorted by name
        content = nc.Dataset(filepath)
        files_list.append(content)                                              # to retrieve netcdf original ATTRIBUTES

ds_hist = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm = nc.Dataset(filepath).source_id)
    ds_hist.append(content)

for i, item in enumerate(files_list):
    if files_list[i].source_id == "CESM2-WACCM":
        ds_hist[i] = ds_hist[i]*-1
        ds_hist[i] = ds_hist[i].assign_attrs(esm = files_list[i].source_id)

##### ------- UNIFORM CALENDAR ------- #####
for i, item in enumerate(ds_hist):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist[i]['time'] = item.indexes['time'].to_datetimeindex()

# -- Resample from Months to Seasons
ds_hist_seas = []
for i,item in enumerate(ds_hist):
    content = item.resample(time="QS", label='left').mean()
    ds_hist_seas.append(content)

ds_hist_seas = [a.sel(lat=slice(-30,30), lon= slice(260,340)) for a in ds_hist_seas]

In [9]:
data_path = 'F:/Data/analysis/'              

scenario = 'ssp585'
var_name = 'nep'
files = var_name + '_*_ssp585_'  + '*.nc' 


files_list = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):               # List of files sorted by name
        content = nc.Dataset(filepath)
        files_list.append(content)                                              # to retrieve netcdf original ATTRIBUTES

ds_ssp = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm = nc.Dataset(filepath).source_id)
    ds_ssp.append(content)

for i, item in enumerate(files_list):
    if files_list[i].source_id == "CESM2-WACCM":
        ds_ssp[i] = ds_ssp[i]*-1
        ds_ssp[i] = ds_ssp[i].assign_attrs(esm = files_list[i].source_id)

##### ------- UNIFORM CALENDAR ------- #####
for i, item in enumerate(ds_ssp):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp[i]['time'] = item.indexes['time'].to_datetimeindex()

# -- Resample from Months to Seasons
ds_ssp_seas = []
for i,item in enumerate(ds_ssp):
    content = item.resample(time="QS", label='left').mean()
    ds_ssp_seas.append(content)

ds_ssp_seas = [a.sel(lat=slice(-30,30), lon= slice(260,340)) for a in ds_ssp_seas]

#### RAD simulations (net)

In [10]:
scenario = 'ssp585-rad'
files = var_name + '_*_ssp585_'  + '*_net.nc' 

ds_ssp_net = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    ds_ssp_net.append(content)                                                                 

##### ------- UNIFORM CALENDAR ------- #####
for i, item in enumerate(ds_ssp_net):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_net[i]['time'] = item.indexes['time'].to_datetimeindex()

# -- Resample from Months to Seasons
ds_ssp_net_seas = []
for i,item in enumerate(ds_ssp_net):
    content = item.resample(time="QS", label='left').mean()
    ds_ssp_net_seas.append(content)
    
# ds_ssp_net_seas = []
# for i, item in enumerate(ds_ssp_net):
#     content = item.rolling(time = 6).mean()                # Compute 7 months running mean
#     content1 = content[content.time.dt.month == 3]     # Select March to get the average over SONDJFM
#     ds_ssp_net_seas.append(content1)

# CanESM5 hist and ssp have numeric values of ocean equal to 
# hist: -9.304011e-07
# ssp: -1.120609e-05

for i, item in enumerate(ds_ssp_net_seas):
    if ds_ssp_net_seas[i].esm == "CanESM5":
        ds_ssp_net_seas[i] = ds_ssp_net_seas[i].where(ds_ssp_net_seas[i] != -3.7270379e-07)
        ds_ssp_net_seas[i] = ds_ssp_net_seas[i].where(ds_ssp_net_seas[i] != -6.18386321e-06)

In [11]:
scenario = 'historical-rad'
files = var_name + '_*_historical_'  + '*_net.nc' 

ds_hist_net = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    ds_hist_net.append(content)                                                                 

##### ------- UNIFORM CALENDAR ------- #####
for i, item in enumerate(ds_hist_net):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_net[i]['time'] = item.indexes['time'].to_datetimeindex()

# -- Resample from Months to Seasons
ds_hist_net_seas = []
for i,item in enumerate(ds_hist_net):
    content = item.resample(time="QS", label='left').mean()
    ds_hist_net_seas.append(content)
    
# ds_hist_net_seas = []
# for i, item in enumerate(ds_hist_net):
#     content = item.rolling(time = 6).mean()                # Compute 7 months running mean
#     content1 = content[content.time.dt.month == 3]     # Select March to get the average over SONDJFM
#     ds_hist_net_seas.append(content1)

# CanESM5 hist and hist have numeric values of ocean equal to 
# hist: -9.304011e-07
# hist: -1.120609e-05

for i, item in enumerate(ds_hist_net_seas):
    if ds_hist_net_seas[i].esm == "CanESM5":
        ds_hist_net_seas[i] = ds_hist_net_seas[i].where(ds_hist_net_seas[i] != -3.7270379e-07)
        ds_hist_net_seas[i] = ds_hist_net_seas[i].where(ds_hist_net_seas[i] != -6.18386321e-06)

In [12]:
ds_hist_seas = [item for item in ds_hist_seas if item.esm not in {"BCC-CSM2-MR", "E3SM-1-1-ECA", "TaiESM1", "NorESM2-MM"}]
ds_ssp_seas = [item for item in ds_ssp_seas if item.esm not in {"BCC-CSM2-MR", "E3SM-1-1-ECA", "TaiESM1", "NorESM2-MM"}]

In [47]:
esm = [item for item in esm if item not in {"BCC-CSM2-MR", "E3SM-1-1-ECA", "TaiESM1", "NorESM2-MM"}]

## Composites analysis

Define Nino events as those DJF seasons lying above 90th perc\
Define Nina events as those DJF seasons lying below 10th perc\
Define Mean State conditions as those DJF seasons in between

Percentiles calculated in the historical scenario (1901-1960) and in the ssp scenario (2040-2100) so that to have the same number of events\

Historical and Future Anomalies are computed with respect to the 1901-1960 and 2041-2100 climatologies respectively (these calculated excluding ENSO events of high magnitude (Nino and Nina events)). \
ENSO effect is computed considering the difference between the Future DJF anomalies with respect to the Historical DJF anomalies.\
MEAN STATE CHANGE is computed considering the difference between Future NEUTRAL conditions and Historical NEUTRAL conditions

In [43]:
seas = "DJF"

def composite_analysis(ds, ds_sst, quant1, quant2, time_init, time_end):
    perc90 = ds_sst.sel(time = ds_sst.time.dt.season==seas).sel(time = slice(time_init,time_end)).quantile(quant1, dim = 'time')      # Percentile only from the DJF season
    perc10 = ds_sst.sel(time = ds_sst.time.dt.season==seas).sel(time = slice(time_init,time_end)).quantile(quant2, dim = 'time')
    # perc90 = ds_sst.sel(time = slice(time_init,time_end)).quantile(quant1, dim = 'time')      # Percentile only from the SONDJFM season
    # perc10 = ds_sst.sel(time = slice(time_init,time_end)).quantile(quant2, dim = 'time')
    nino_events = ds.sel(time = ds.time.dt.season==seas).sel(time = slice(time_init,time_end)).where(ds_sst > perc90, drop=True)
    nina_events = ds.sel(time = ds.time.dt.season==seas).sel(time = slice(time_init,time_end)).where(ds_sst < perc10, drop=True)
    # nino_events = ds.sel(time = slice(time_init,time_end)).where(ds_sst > perc90, drop=True)
    # nina_events = ds.sel(time = slice(time_init,time_end)).where(ds_sst < perc10, drop=True)
    ds_nino = ds.sel(time = nino_events.time, method ="nearest").sel(time = slice(time_init,time_end))  
    ds_nina = ds.sel(time = nina_events.time, method ="nearest").sel(time = slice(time_init,time_end))          
    # ds_nino = ds_nino.mean(dim = "time")        # Composites of El Nino
    # ds_nina = ds_nina.mean(dim = "time")        # Composites of La Nina
    content = ds.sel(time = ds.time.dt.season==seas).sel(time=slice(time_init,time_end))
    # content = ds.sel(time=slice(time_init,time_end))
    content1 = nino_events.groupby(nino_events.time.dt.year).mean("time")  # NINO events between 1901 an6 1910
    content2 = nina_events.groupby(nina_events.time.dt.year).mean("time")  # NINO events between 1901 an6 1910
    for ninoyrs in np.array(content1.year):                                                                            # drop NINO events'years recursively
        content = content.where(content.time.dt.year != ninoyrs, drop = True)
    for ninayrs in np.array(content2.year):
        content = content.where(content.time.dt.year != ninayrs, drop = True)                                          # drop NINO events'years recursively   
    ds_clim_neutral = content #.mean(dim="time")  # Composites of mean state changes (Neutral years)

    ds_nino = ds_nino.assign_attrs(esm = ds.esm)
    ds_nina = ds_nina.assign_attrs(esm = ds.esm)
    ds_clim_neutral = ds_clim_neutral.assign_attrs(esm = ds.esm)
    return ds_clim_neutral, ds_nino, ds_nina

results_hist = [composite_analysis(a, b, 0.9, 0.1, "1901", "1960") for a,b in zip(ds_hist_seas, nino34_hist_seas)]
ds_clim_neutral_hist_nep = [result[0] for result in results_hist]
ds_nino_hist_nep = [result[1] for result in results_hist]
ds_nina_hist_nep = [result[2] for result in results_hist]

results_ssp = [composite_analysis(a, b, 0.9, 0.1, "2041", "2100") for a,b in zip(ds_ssp_seas, nino34_ssp_seas)]
ds_clim_neutral_ssp_nep = [result[0] for result in results_ssp]
ds_nino_ssp_nep = [result[1] for result in results_ssp]
ds_nina_ssp_nep = [result[2] for result in results_ssp]

results_eco2 = [composite_analysis(a, b, 0.9, 0.1, "2041", "2100") for a,b in zip(ds_ssp_net_seas, nino34_ssp_seas)]
ds_clim_neutral_ssp_nep_net = [result[0] for result in results_eco2]

# For MannWhitneyU calculation, the significance is calculated among a composite for SSP (eg (time=12,lon=30,lat=37)) and HIST (eg (time=5,lon=30,lat=37))
# For Anomalies calculation , the anomalies are calculated among the average of those SSP and HIST samples.
# As a consequence, in the code above we need to take the mean along time dimension of the followings:
##  ds_ssp_seas_clim_allseason
##  ds_ssp_seas_clim_nonino
##  ds_ssp_seas_clim_nino

## -- NINO HISTORICAL ANOMALIES
nino_anom_hist = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(ds_nino_hist_nep, ds_clim_neutral_hist_nep)]

## -- NINO SSP585 ANOMALIES
nino_anom_ssp = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(ds_nino_ssp_nep, ds_clim_neutral_ssp_nep)]

## -- NINO EFFECT (considering the difference between anomalies)
nino_effect = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(nino_anom_ssp, nino_anom_hist)]

## -- NINA HISTORICAL ANOMALIES
nina_anom_hist = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(ds_nina_hist_nep, ds_clim_neutral_hist_nep)]

## -- NINA SSP585 ANOMALIES
nina_anom_ssp = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(ds_nina_ssp_nep, ds_clim_neutral_ssp_nep)]

## -- NINA EFFECT (considering the difference between anomalies)
nina_effect = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(nina_anom_ssp, nina_anom_hist)]

## -- MEAN STATE CHANGE
msc_tot = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(ds_clim_neutral_ssp_nep, ds_clim_neutral_hist_nep)]

## -- MSC net
msc_net = [(a - b).compute(dim=var_name).assign_attrs(esm = a.esm) for a,b in zip(ds_clim_neutral_ssp_nep_net, ds_clim_neutral_hist_nep)]


## Significance Test (Mann-Whitney)
### Control for False Discovery Rate (FDR) in multiple testing procedure

In [49]:
#### --------- Mann-Whitney Test --------- ####
from scipy.stats import mannwhitneyu

# Test applied on a grid-cell basis. For every gid-cell, the statistical difference between two time series is computed.
# 1st time series: Nino years' (ds_hist_nino)
# 2nd time series: Reference climatology (ds_hist_clim or ds_hist_clim_neutral)  

## -- Function for grid-cell operations -- ##
def multi_apply_along_axis(func1d, axis, arrs, *args, **kwargs):
    """
    Given a function `func1d(A, B, C, ..., *args, **kwargs)`  that acts on 
    multiple one dimensional arrays, apply that function to the N-dimensional
    arrays listed by `arrs` along axis `axis`
    
    If `arrs` are one dimensional this is equivalent to::
    
        func1d(*arrs, *args, **kwargs)
    
    If there is only one array in `arrs` this is equivalent to::
    
        numpy.apply_along_axis(func1d, axis, arrs[0], *args, **kwargs)
        
    All arrays in `arrs` must have compatible dimensions to be able to run
    `numpy.concatenate(arrs, axis)`
    
    Arguments:
        func1d:   Function that operates on `len(arrs)` 1 dimensional arrays,
                  with signature `f(*arrs, *args, **kwargs)`
        axis:     Axis of all `arrs` to apply the function along
        arrs:     Iterable of numpy arrays
        *args:    Passed to func1d after array arguments
        **kwargs: Passed to func1d as keyword arguments
    """
    # Concatenate the input arrays along the calculation axis to make one big
    # array that can be passed in to `apply_along_axis`
    carrs = np.concatenate(arrs, axis)
    
    # We'll need to split the concatenated arrays up before we apply `func1d`,
    # here's the offsets to split them back into the originals
    offsets=[]
    start=0
    for i in range(len(arrs)-1):
        start += arrs[i].shape[axis]
        offsets.append(start)
            
    # The helper closure splits up the concatenated array back into the components of `arrs`
    # and then runs `func1d` on them
    def helperfunc(a, *args, **kwargs):
        arrs = np.split(a, offsets)
        return func1d(*[*arrs, *args], **kwargs)
    
    # Run `apply_along_axis` along the concatenated array
    return np.apply_along_axis(helperfunc, axis, carrs, *args, **kwargs)


def xr_multipletest(p, alpha=0.05, method='fdr_bh', **multipletests_kwargs):
    """Apply statsmodels.stats.multitest.multipletests for multi-dimensional xr.objects."""
    from statsmodels.stats.multitest import multipletests
    # stack all to 1d array
    p_stacked = p.stack(s=p.dims)
    # mask only where not nan: https://github.com/statsmodels/statsmodels/issues/2899
    mask = np.isfinite(p_stacked)
    pvals_corrected = np.full(p_stacked.shape, np.nan)
    reject = np.full(p_stacked.shape, np.nan)
    # apply test where mask
    reject[mask] = multipletests(
        p_stacked[mask], alpha=alpha, method=method, **multipletests_kwargs)[0]
    pvals_corrected[mask] = multipletests(
        p_stacked[mask], alpha=alpha, method=method, **multipletests_kwargs)[1]

    def unstack(reject, p_stacked):
        """Exchange values from p_stacked with reject (1d array) and unstack."""
        xreject = p_stacked.copy()
        xreject.values = reject
        xreject = xreject.unstack()
        return xreject

    reject = unstack(reject, p_stacked)
    pvals_corrected = unstack(pvals_corrected, p_stacked)
    return reject, pvals_corrected


#### MW test with FDR correction

In [50]:
## ----- MW Test on every MODEL with FDR correction ----- ##
# Obtain dim: nmodels*2*(lon*lat)
# ds_hist_seas_nino & ds_hist_seas_clim_neutral ---> MannWhitney_historical
# ds_ssp_seas_nino & ds_ssp_seas_clim_neutral ---> MannWhitney_ssp
# ds_ssp_seas_clim_neutral & ds_hist_seas_clim_neutral ---> MannWhitney_ssp_msc
# ds_ssp_seas_nino & ds_hist_seas_nino ---> MannWhitney_ssp_nino

mannwhitney_msc=[]
mannwhitney_msc_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_clim_neutral_ssp_nep_net[i].fillna(0), ds_clim_neutral_hist_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_clim_neutral_ssp_nep_net[i].lon.values), lat= (ds_clim_neutral_hist_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_msc.append(mw)
    mannwhitney_msc_fdr.append(mw_fdr)

mannwhitney_nino_effect=[]
mannwhitney_nino_effect_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_nino_ssp_nep[i].fillna(0), ds_nino_hist_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_nino_ssp_nep[i].lon.values), lat= (ds_nino_hist_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_nino_effect.append(mw)
    mannwhitney_nino_effect_fdr.append(mw_fdr)

mannwhitney_nina_effect=[]
mannwhitney_nina_effect_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_nina_ssp_nep[i].fillna(0), ds_nina_hist_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_nina_ssp_nep[i].lon.values), lat= (ds_nina_hist_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_nina_effect.append(mw)
    mannwhitney_nina_effect_fdr.append(mw_fdr)

mannwhitney_nino_anom_hist=[]
mannwhitney_nino_anom_hist_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_nino_hist_nep[i].fillna(0), ds_clim_neutral_hist_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_nino_hist_nep[i].lon.values), lat= (ds_clim_neutral_hist_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_nino_anom_hist.append(mw)
    mannwhitney_nino_anom_hist_fdr.append(mw_fdr)

mannwhitney_nina_anom_hist=[]
mannwhitney_nina_anom_hist_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_nina_hist_nep[i].fillna(0), ds_clim_neutral_hist_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_nina_hist_nep[i].lon.values), lat= (ds_clim_neutral_hist_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_nina_anom_hist.append(mw)
    mannwhitney_nina_anom_hist_fdr.append(mw_fdr)

mannwhitney_nino_anom_ssp=[]
mannwhitney_nino_anom_ssp_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_nino_ssp_nep[i].fillna(0), ds_clim_neutral_ssp_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_nino_ssp_nep[i].lon.values), lat= (ds_clim_neutral_ssp_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_nino_anom_ssp.append(mw)
    mannwhitney_nino_anom_ssp_fdr.append(mw_fdr)

mannwhitney_nina_anom_ssp=[]
mannwhitney_nina_anom_ssp_fdr=[]
for i,item in enumerate(esm):
    mw = multi_apply_along_axis(mannwhitneyu, 0, [ds_nina_ssp_nep[i].fillna(0), ds_clim_neutral_ssp_nep[i].fillna(0)])[1]
    mw_xr = xr.DataArray(mw, dims=["lat","lon"], coords=dict(lon = (ds_nina_ssp_nep[i].lon.values), lat= (ds_clim_neutral_ssp_nep[i].lat.values)))
    mw_fdr = xr_multipletest(mw_xr)
    mannwhitney_nina_anom_ssp.append(mw)
    mannwhitney_nina_anom_ssp_fdr.append(mw_fdr)

In [51]:
## -- Save and export Mann-Whitney list data
import pickle

data_path = 'C:/Users/mastr/Documents/Amazon'

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"msc_net_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_msc_fdr, fp)

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nino_effect_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_nino_effect_fdr, fp)

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nina_effect_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_nina_effect_fdr, fp)

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nino_anom_hist_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_nino_anom_hist_fdr, fp)

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nina_anom_hist_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_nina_anom_hist_fdr, fp)

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nino_anom_ssp_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_nino_anom_ssp_fdr, fp)

with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nina_anom_ssp_fdr"), "wb") as fp:   #Pickling
    pickle.dump(mannwhitney_nina_anom_ssp_fdr, fp)

In [ ]:
# ## -- Save and export Mann-Whitney list data
# import pickle

# data_path = 'C:/Users/mastr/Documents/Amazon'

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"msc"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_msc, fp)

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nino_effect"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_nino_effect, fp)

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nina_effect"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_nina_effect, fp)

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nino_anom_hist"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_nino_anom_hist, fp)

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nina_anom_hist"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_nina_anom_hist, fp)

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nino_anom_ssp"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_nino_anom_ssp, fp)

# with open(os.path.join(data_path+"/analysis/MannWhitney_"+var_name+"_"+"nina_anom_ssp"), "wb") as fp:   #Pickling
#     pickle.dump(mannwhitney_nina_anom_ssp, fp)